# 23 - Iterators and Generators

---

Iterators allow your classes to be used in `for ... in ...` statements. Generators are an easy way to create iterators.

---

## Iterators

You have used the `for ... in ...` command on many occasions. You may have noticed that it can be used for many different applications.

In [ ]:
for i in [1,2,3,4]:
    print( i, end=" " )
print()
for i in ( "pi", 3.14, 22/7 ):
    print( i, end=" ")
print()
for i in range( 3, 11, 2 ):
    print( i, end=" ")
print()
for c in "Hello":
    print( c, end=" " )
print()
for key in { "apple":1, "banana":3 }:
    print( key, end=" " )

List, strings, and dictionaries are all "iterables", which means they can be used in such `for ... in ...` expressions. Many other objects can also be used as iterables. You can actually ensure that your own classes can be used as iterables as well.

An "iterator" is an object that returns a new item every time you call the `next()` function with the object as argument. When there are no items left, it raises a `StopIteration` exception. If you want to avoid the exception, you can give an optional second argument to `next()`, which is returned when the iterator is exhausted. You can turn an iterable into an iterator object using the built-in function `iter()`.

In [ ]:
iterator = iter( ["apple", "banana", "cherry"] )
print( next( iterator, "END" ) )
print( next( iterator, "END" ) )
print( next( iterator, "END" ) )
print( next( iterator, "END" ) )

You can use iterators in `for ... in ...` statements.

In [ ]:
iterator = iter( ["apple", "banana", "cherry"] )
for fruit in iterator:
    print( fruit )

### Iterable objects

An object that should function as an iterable has two elements: 
- a method `__iter__()` that returns the object itself
- a method `__next__()` that provides access to all the items that the object contains, one by one, and when no more objects are left, raises `StopIteration`. 

You can loop over all the items of the iterable using `for ... in ...`. There are three main ways that you can create such an iterable object. The first two ways start with the iterable as a container of a sequence of items. 

The first way, when `__next__()` is called, removes one of the items and returns it, after which the iterable holds one less item. Once all items are "consumed", it can only raise `StopIteration`. Here is an example of such an iterator that contains the first 10 numbers of the Fibonacci sequence.

In [ ]:
class Fibo:
    def __init__( self ):
        self.seq = [1,1,2,3,5,8,13,21,34,55]
    def __iter__( self ):
        return self
    def __next__( self ):
        if len( self.seq ) > 0:
            return self.seq.pop(0)
        raise StopIteration()
        
fseq = Fibo()
for n in fseq:
    print( n, end=" " )

The second way keeps track of an index in the sequence of items, increasing it every time `__next__()` is called, and raises `StopIteration` when the index goes beyond the range of items. In the second way, you can implement a method that resets the index so that the iterable can be used again.

In [ ]:
class Fibo:
    def __init__( self ):
        self.seq = [1,1,2,3,5,8,13,21,34,55]
        self.index = -1
    def __iter__( self ):
        return self
    def __next__( self ):
        if self.index < len( self.seq )-1:
            self.index += 1
            return self.seq[self.index]
        raise StopIteration()
    def reset( self ):
        self.index = -1
        
fseq = Fibo()
for n in fseq:
    print( n, end=" " )
print()
fseq.reset()
for n in fseq:
    print( n, end=" " )

The third way has the iterable not as a container of items, but as a calculator that every time that `__next__()` is called, determines the next item. Such an iterable can either be finite, or have the ability to supply an infinite number of items. It can also be reset if the programmer supplied a method to do that.

In [ ]:
class Fibo:
    def reset( self ):
        self.nr1 = 0
        self.nr2 = 1
    def __init__( self, maxnum=1000 ):
        self.maxnum = maxnum
        self.reset()
    def __iter__( self ):
        return self
    def __next__( self ):
        if self.nr2 > self.maxnum:
            raise StopIteration()
        nr3 = self.nr1 + self.nr2
        self.nr1 = self.nr2
        self.nr2 = nr3
        return self.nr1
        
fseq = Fibo()
for n in fseq:
    print( n, end=" " )
print()
fseq.reset()
for n in fseq:
    print( n, end=" " )

Of course, you have to be very careful when making an iterable that in principle may return an infinite number of items. Programmers count on `for ... in ...` never leading to an endless loop, but in the example above, without limiting the number of items to a maximum of 1000 when creating the `fseq` object, an endless loop would result. It is best to force the programmer to set a maximum to the number of items.

**Exercise**: In the code block below, create an iterator that generates all the squares of integers between 1 and 10. You may choose whichever approach you prefer.

In [ ]:
# Squares iterator.


### Delegated iteration

In the examples above, the iterable was created by calling the `__iter__()` method for the object, which returned itself. That is not needed. An iterable may delegate the iteration to another object, that it creates and returns when `__iter__()` is called.

In [ ]:
class FiboIterable:
    def __init__( self, seq ):
        self.seq = seq
    def __next__( self ):
        if len( self.seq ) > 0:
            return self.seq.pop(0)
        raise StopIteration()

class Fibo:
    def __init__( self, maxnum=1000 ):
        self.maxnum = maxnum
    def __iter__( self ):
        nr1 = 0
        nr2 = 1
        seq = []
        while nr2 <= self.maxnum:
            nr3 = nr1 + nr2
            nr1 = nr2
            nr2 = nr3
            seq.append( nr1 )
        return FiboIterable( seq )
        
fseq = Fibo()
for n in fseq:
    print( n, end=" " )
print()
for n in fseq:
    print( n, end=" " )

This approach has several advantages:
- You can run several instances of the iterable in parallel without the need to explicitly create more than one (as they are created automatically when needed, i.e., when you use `for ... in ...`)
- You do not need to call a `reset()` method to start from the beginning
- The delegated iterable is automatically erased from memory after it is used up (Python automatically frees up memory of objects that the program no longer can refer to)

Note: The name "delegated iterable" I came up with myself. If there is an "official" name for the approach, I gladly update the notebook.

### `zip()`

You can create tuples that contain the items of multiple iterables using the standard function `zip()`. To give a simple example:

In [ ]:
z = zip( [1,2,3], [4,5,6], [7,8,9] )
for x in z:
    print( x )

A zip-object is an iterator, i.e., you cannot print the zip-object itself, but you have to loop over its elements instead. The `i`th element of the zip-object consists of the `i`th elements of each of the iterables that are its arguments. If these iterables are of unequal length, the number of elements in the zip-object will be the same as the number of elements of the shortest of the iterables.

In the example above, I just used lists as arguments. But you can use any kind of iterable as argument. For example, the following code block I zip together a range, and iterator, and a list comprehension:

In [ ]:
class Doubles:
    def __init__( self ):
        self.seq = [2*x for x in range( 1, 11 )]
    def __iter__( self ):
        return self
    def __next__( self ):
        return self.seq.pop(0)
    
seq = zip( range( 1, 11 ), Doubles(), [3*x for x in range( 1, 11 )] )
for x in seq:
    print( x )

**Exercise**: Create a zip-object that produces tuples of two items: the first item is an integer, which runs from 1 to 10. The second item is the square of that integer.

In [ ]:
# Squares tuples.


### `reversed()`

The built-in function `reversed()` creates an iterator from an iterable that processes the items of the iterator in reversed order. It gets the iterable as argument. Not all iterables can be reversed, but the ones that are part of the standard Python specification (such as lists) can. For details on how to make sure that iterables that you create yourself can be `reversed()`, study the Python documentation.

In [ ]:
fruitlist = ["apple", "orange", "cherry", "banana"]
for fruit in reversed( fruitlist ):
    print( fruit )

### `sorted()`

The built-in function `sorted()` creates an iterator from an iterable that processes the items of the iterator in sorted order. It gets the iterable as argument. Moreover, it can get two optional arguments. The first is `key=<key>`, where `<key>` is the name of a function that is used to determine the key for the sorting process. This works exactly as the `key=<key>` parameter for the list `sort()` method -- see the chapter on lists for more information. If no key is given, the sorting is alphabetical order for strings, and numerical order for numbers. For other data types, or mixed data types, it depends on the specification of the `key` argument. The second optional argument is `reverse=<boolean>`, that indicates with `True` or `False` whether or not the sorting should give a reversed result.

In [ ]:
fruitlist = ["apple", "orange", "cherry", "banana"]
for fruit in sorted( fruitlist ):
    print( fruit )

---

## Generators

A generator is a function that emulates the behavior of an iterable object. In general, implementing a generator is shorter and easier than creating an iterable.

Generators are based on the `yield` keyword. When calling `__next__()` on a generator, the function is executed until `yield` is reached, then the value that is associated with `yield` is returned. At that point, the function "waits" until `__next__()` is called again, after which it continues until `yield` is reached again. `StopIteration` is raised automatically when the function ends.

There is no need to explicitly define `__next__()` and/or `__iter__()`. A function is a generator simply because it contains the `yield` keyword, and the associated iterable object is automatically created by Python, including appropriate implementations for `__next__()` and `__iter__()`.

In [ ]:
def fibo( maxnum ):
    nr1 = 0
    nr2 = 1
    while nr2 <= maxnum:
        nr3 = nr1 + nr2
        nr1 = nr2
        nr2 = nr3
        yield nr1
        
fseq = fibo( 1000 )
for n in fseq:
    print( n, end=" " )
print()
for n in fseq:
    print( n, end=" " )

Note that several standard functions are implemented as generators, for example `range()`.

### Generator expressions

In the chapter on lists, I introduced the concept of list comprehension. Since any list can be turned into an iterator, and thus into a generator, Python introduced a similar concept for generators, and calls it "generator expressions". The syntax for a generator expression is the same as for a list comprehension, except that the square brackets are replaced by round brackets.

For example, the following generator expression returns all squares up to 100:

In [ ]:
seq = (x*x for x in range( 11 ))
for x in seq:
    print( x, end=" " )

If you just replace the outer two parentheses by square brackets in the generator expression, the code runs with `seq` being the result of list comprehension. To be absolutely clear about it: with list comprehension the whole list is generated at once, while with a generator expression the items are generated when needed. Thus, in principle a generator expression is preferable, as it saves memory.

---

## `itertools` module

The `itertools` module contains a collection of functions that allow advanced manipulation of iterators. Taken to the extreme, they allow for a sort of "iterator algebra" that can be used to implement specialized tools in Python. Here I just highlight a few of the basic functions from `itertools` that you might find handy at times.

### `chain()`

`chain()` takes two or more iterables as arguments and functions as an iterable that works through them in sequence.

In [ ]:
from itertools import chain

seq = chain( [1,2,3], [11,12,13,14,15], [x*x for x in range( 1, 6 )] )
for item in seq:
    print( item, end=" ")

### `zip_longest()`

`zip_longest()` works like `zip()`, but will create an iterable that generates as many elements as there are elements in the longest argument. You specify a `fillvalue=` argument to indicate what value should be used for empty spots.

In [ ]:
from itertools import zip_longest

seq = zip_longest( "apple", "strawberry", "banana", fillvalue=" " )
for item in seq:
    print( item )

### `product()`

`product()` creates an iterable that produces all elements of the Cartesian product of the iterables that are given as its arguments. To put that in less mathematical terms: if two iterables are given as arguments, and the first has elements `x`, `y`, and `z`, while the second has elements `a` and `b`, `product()` produces `xa`, `xb`, `ya`, `yb`, `za`, and `zb`.

In [ ]:
from itertools import product

seq = product( [1,2,3], "ABC", ["apple","banana"] )
for item in seq:
    print( item )

### `permutations()`

`permutations()` gets an iterable as argument, and an optional second argument that indicates a length. It creates an iterable that produces all permutations of the elements of the first argument of the given length. If no length is given, it generates all permutations that contain all the elements. Note that if the iterable has certain elements multiple times, you will get copies of permutations.

In [ ]:
from itertools import permutations

seq = permutations( [1,2,3], 2 )
for item in seq:
    print( item )

### `combinations()`

`combinations()` gets an iterable as argument, and a second argument that indicates a length. It creates an iterable that produces all combinations of the elements of the first argument of the given length. The length is *not* optional (which is logical, if you think about it for one moment). The elements of the combinations will be in the order that they appeared in the original iterable. Note that if the iterable has certain elements multiple times, you will get copies of combinations.

In [ ]:
from itertools import combinations

seq = combinations( [1,2,3], 2 )
for item in seq:
    print( item )

### `combinations_with_replacement()`

`combinations_with_replacement()` works like `combinations()`, except that each element of the iterable can be used multiple times.

In [ ]:
from itertools import combinations_with_replacement

seq = combinations_with_replacement( [1,2,3], 2 )
for item in seq:
    print( item )

---

## What you learned

In this chapter, you learned about:

- Iterators
- Iterables
- `__iter__()`, `__next__()`, and `StopIteration`
- Different approaches to implementing iterable objects
- `zip()`
- Generators
- `yield`
- Generator expressions
- From `itertools` the functions `chain()`, `zip_longest()`, `product()`, `permutations()`, `combinations()`, and `combinations_with_replacement()`

---

## Exercises

### Exercise 23.1

Create a program that asks the user to enter positive integers. The user can enter as many as desired, and indicates that the last integer was entered by supplying zero. The program the prints all numbers between 1 and 100 that are not dividable by any of the integers entered. Print those numbers in a `for ... in ...` loop, using an iterator to produce the numbers. 

In [ ]:
# Not dividable by.


### Exercise 23.2

Create a generator that produces factorials. The first value returned is `1!`, the second `2!`, the third `3!`, etcetera, up to `10!`. Do not calculate the factorial every time from scratch, but retain the value that you used in the previous cycle and use that.

In [ ]:
# Factorial generator.


### Exercise 23.3

Ask the user to enter a word. Produce all anagrams of that word. If the word contains multiple copies of a letter, it is acceptable if you produce certain anagrams multiple times. For example, if the word is "ape", you produce "aep", "ape", "eap", "epa", "pae", and "pea" (in any order).

In [ ]:
# Anagrams.


### Exercise 23.4

Do exercise 23.3, but now make sure that all anagrams are unique, even if the word contains repetitions of letters. For example, if the word is "bee", you produce "bee", "ebe", and "eeb".

In [ ]:
# Unique anagrams.


### Exercise 23.5

The "subset sum" problem asks the question whether a list of integers contains a subset of integers that, when summed, gives zero as answer. For instance, for the list `[1, 4, -3, -5, 7]` the answer is "yes", as `1 + 4 - 5 = 0`. However, for the list `[1, 4, -3, 7]` the answer is "no", as there is no subset of integers that adds up to zero. Write a program that solves the "subset sum" problem for a list of integers. If there is a solution, print it; if not, report that there is no solution.

You might remember that you saw this problem description before. It is a repetition of exercise 13.8, from the chapter on lists. In that chapter I said that you have to solve the exercise recursively. However, using the `itertools` module, you can now solve it without recursion (though I suspect that recursion still is used within the `itertools` module -- you, however, do not have to).

In [ ]:
# Subset sum problem without recursion.


---

## Python 2

Python 3 is much more based on iterators and generators than Python 2, which has all kinds of advantages, mainly as far as speed and memory usage is concerned. Consequently, there are quite a few differences between Python 2 and 3 in this respect. I have not inventoried all of them, but here is a list of the ones that struck me:

Iterators in Python 2 have a `next()` method. They no longer have it in Python 3, where it is called `__next__()`. 

In Python 2 `zip()` produces a list rather than an iterable. 

The `itertools` module has some differences too. For instance, in Python 2 it has a function `izip()` that produces an iterable, but since in Python 3 `zip()` does that already, the function `izip()` has been removed from `itertools`. 

---

End of Chapter 23. Version 1.0. 